In [50]:
import sys
import os
from dotenv import load_dotenv
import pandas as pd

# Add the parent directory to the sys.path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from sqltoolkit.connectors import PostgreSQLConnector
from sqltoolkit.client import DatabaseClient
from sqltoolkit.entities import Table, TableColumn

load_dotenv()

# Define the connection parameters
server = os.getenv('POSTGRES_HOST')
database = 'debit_card_specializing'
username = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PWD')
port = '5432'

psql_connector = PostgreSQLConnector(server, database, username, password, port)
sql_client = DatabaseClient(psql_connector)

In [51]:
from openai import AzureOpenAI

aoai_endpoint = os.getenv('OPENAI_ENDPOINT')
aoai_key = os.getenv('OPENAI_API_KEY')
aoai_deployment = os.getenv('OPENAI_4o_DEPLOYMENT') # should be GPT-4o
aoai_embedding_deployment = os.getenv('OPENAI_EMBEDDING_DEPLOYMENT') # should be text-embedding-small

openai_client = AzureOpenAI(azure_endpoint=aoai_endpoint,
    api_key=aoai_key,
    api_version='2024-10-21')

In [3]:
import os
import pandas as pd

directory = 'debit_card_specializing_doc'
concatenated_content = ""

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        table_name = filename.split('.')[0]
        df = pd.read_csv(os.path.join(directory, filename))
        table_markdown = df.to_markdown(index=False)
        concatenated_content += f"\n### Table: {table_name}\n\n{table_markdown}\n\n"

print(concatenated_content)


### Table: transactions_1k

| original_column_name   | column_name    | column_description   | data_format   | value_description            |
|:-----------------------|:---------------|:---------------------|:--------------|:-----------------------------|
| TransactionID          | Transaction ID | Transaction ID       | integer       | nan                          |
| Date                   | nan            | Date                 | date          | nan                          |
| Time                   | nan            | Time                 | text          | nan                          |
| CustomerID             | Customer ID    | Customer ID          | integer       | nan                          |
| CardID                 | Card ID        | Card ID              | integer       | nan                          |
| GasStationID           | Gas Station ID | Gas Station ID       | integer       | nan                          |
| ProductID              | Product ID     | Product ID     

In [4]:
from sqltoolkit.indexer import DatabaseIndexer

context = concatenated_content

indexer = DatabaseIndexer(sql_client, 
                          openai_client, 
                          aoai_deployment,
                          aoai_embedding_deployment,
                          extra_context=context)

manifest = indexer.fetch_and_describe_tables()
indexer.generate_table_embeddings()

tables_dict = indexer.export_json_manifest()

2024-12-18 13:06:15,248 - DatabaseIndexer - INFO - Fetching tables from the database.
2024-12-18 13:06:15,433 - DatabaseIndexer - INFO - Processing table: public.customers
2024-12-18 13:06:20,312 - DatabaseIndexer - INFO - Completed processing table: public.customers
2024-12-18 13:06:20,314 - DatabaseIndexer - INFO - Processing table: public.gasstations
2024-12-18 13:06:27,721 - DatabaseIndexer - INFO - Completed processing table: public.gasstations
2024-12-18 13:06:27,723 - DatabaseIndexer - INFO - Processing table: public.products
2024-12-18 13:06:31,181 - DatabaseIndexer - INFO - Completed processing table: public.products
2024-12-18 13:06:31,182 - DatabaseIndexer - INFO - Processing table: public.transactions_1k


Object of type date is not JSON serializable


2024-12-18 13:06:44,404 - DatabaseIndexer - INFO - Completed processing table: public.transactions_1k
2024-12-18 13:06:44,405 - DatabaseIndexer - INFO - Processing table: public.yearmonth
2024-12-18 13:06:49,301 - DatabaseIndexer - INFO - Completed processing table: public.yearmonth
2024-12-18 13:06:49,302 - DatabaseIndexer - INFO - Completed fetching and processing all tables.


In [9]:
indexer.tables[0].model_dump()

{'name': 'public.customers',
 'business_readable_name': 'Customer Segments',
 'description': 'The customers table contains information about individual customers, including a unique identifier, their client segment classification, and the currency they use for transactions. This table helps in distinguishing customers and categorizes them into different groups such as Key Account Management (KAM), Large Account Management (LAM), and Small and Medium Enterprises (SME). The currency column indicates the currency codes like CZK and EUR used by customers in their transactions. This information is used for tailoring services and performing financial analyses.',
 'columns': [{'name': 'customerid',
   'type': 'bigint',
   'description': None,
   'definition': 'Customer ID is a unique identifier assigned to each customer in the database. It is represented as a bigint integer and serves as the primary key for the customers table. This column helps in distinguishing individual customers and is u

In [10]:
search_endpoint = os.getenv('AI_SEARCH_ENDPOINT')
search_key = os.getenv('AI_SEARCH_API_KEY')
embedding_deployment = os.getenv('OPENAI_EMBEDDING_DEPLOYMENT')
index_name = database

indexer.create_azure_ai_search_index(
    search_endpoint=search_endpoint,
    search_credential=search_key,
    index_name=index_name,
    embedding_deployment=aoai_embedding_deployment,
    openai_endpoint=aoai_endpoint,
    openai_key=aoai_key,
)

# write to AI Search
indexer.push_to_ai_search()

2024-12-18 13:11:11,603 - DatabaseIndexer - INFO - Creating Azure AI Search Index.
2024-12-18 13:11:11,745 - DatabaseIndexer - INFO - SearchIndexClient created.
2024-12-18 13:11:11,983 - DatabaseIndexer - INFO - Index 'debit_card_specializing' does not exist. Creating a new one.
2024-12-18 13:11:11,985 - DatabaseIndexer - INFO - Fields for the index defined.
2024-12-18 13:11:11,985 - DatabaseIndexer - INFO - Vector search configuration defined.
2024-12-18 13:11:11,986 - DatabaseIndexer - INFO - Semantic configuration defined.
2024-12-18 13:11:11,988 - DatabaseIndexer - INFO - Semantic search configuration defined.
2024-12-18 13:11:12,515 - DatabaseIndexer - INFO - Index 'debit_card_specializing' created successfully.
2024-12-18 13:11:12,519 - DatabaseIndexer - INFO - Pushing metadata for 5 tables to Azure AI Search.


Pushing data for table public.customers to the index.


2024-12-18 13:11:12,796 - DatabaseIndexer - INFO - Data for table public.customers pushed to the index.
2024-12-18 13:11:12,866 - DatabaseIndexer - INFO - Data for table public.gasstations pushed to the index.
2024-12-18 13:11:12,935 - DatabaseIndexer - INFO - Data for table public.products pushed to the index.


Pushing data for table public.gasstations to the index.
Pushing data for table public.products to the index.
Pushing data for table public.transactions_1k to the index.


2024-12-18 13:11:13,004 - DatabaseIndexer - INFO - Data for table public.transactions_1k pushed to the index.
2024-12-18 13:11:13,067 - DatabaseIndexer - INFO - Data for table public.yearmonth pushed to the index.


Pushing data for table public.yearmonth to the index.


In [ ]:
import time
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential
import json

search_client = SearchClient(endpoint=search_endpoint, 
                             credential=AzureKeyCredential(search_key), 
                             index_name=index_name)

def execute_query_pipeline(user_question, evidence=None):
    # Query rewriting
    rewriter_prompt = f"""
    You are a helpful AI SQL Agent. Your role is to help people translate their questions 
    into better questions that can be translated into SQL queries.
    You should disambiguate the question and provide a more specific version of the question to the best of your ability.
    You should rewrite the question in a way that is more likely to be answered by the database.

    ## You can use the following context to help you rewrite the question:
    {evidence}

    You must only return a single sentence that is a better version of the user's question.
    """

    messages = [
        {
            "role": "system",
            "content": rewriter_prompt
        },
        {
            "role": "user",
            "content": user_question
        }
    ]

    rewriter_response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )

    rewriter_response = rewriter_response.choices[0].message.content
    print("Rewritten Question:", rewriter_response)

    # AI Search
    vector_query = VectorizableTextQuery(text=rewriter_response, k_nearest_neighbors=50, fields="embedding")

    results = search_client.search(
        search_text=user_question,
        vector_queries=[vector_query],
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name='my-semantic-config',
        query_caption=QueryCaptionType.EXTRACTIVE,
        query_answer=QueryAnswerType.EXTRACTIVE,
        top=5
    )

    candidate_tables = []
    candidate_table_names = []
    for result in results:
        candidate_tables.append(
            json.dumps({key: result[key] for key in ['name', 'business_readable_name', 'description', 'columns'] if key in result}, indent=4)
        )
        candidate_table_names.append(result['name'])

    # LLM Call
    print("Candidate Tables:", candidate_tables)
    tables_prompt = candidate_tables

    system_prompt = f"""
    You are a helpful AI data analyst assistant, 
    You can execute SQL queries to retrieve information from a SQL database,
    The database is SQL server, use the right syntax to generate queries

    ### You must use the POSTGRES SQL syntax to query the database
    ### You must use double quotes around the column names but not around table names
    ### Your output must be a single SQL query
    ### You must use the column sample values to infer the value in your WHERE clause

    ### Here's additional context to help you generate the SQL query:
    {evidence}

    ### These are the available tables in the database alongside their descriptions:
    {tables_prompt}

    When asked a question that could be answered with a SQL query: 
    - Break down the question into smaller parts that can be answered with SQL queries
    - Identify the tables that contain the information needed to answer the question
    - Only once this is done, create a SQL query to retrieve the information based on your understanding of the table

    # you must use the exact table names and column names provided in the table descriptions

    Think step by step, before doing anything, share the different steps you'll execute to get the answer

    Your response should be in JSON format with the following structure:
    "chain_of_thought": "Your reasoning",
    "sql_query": "The generated SQL query, this can only be a single query"
    """

    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_question
        }
    ]

    response = openai_client.chat.completions.create(
        model="gpt-4o-global",
        messages=messages,
        response_format={"type": "json_object"}
    )

    response_message = response.choices[0].message.content
    response_json = json.loads(response_message)
    print("Generated Chain of Thought:", response_json['chain_of_thought'])
    print("Generated SQL Query:", response_json['sql_query'])

    # Execute SQL Query
    # query_result = sql_client.query(response_json['sql_query'])
    return {'sql_query':response_json['sql_query'],
            'chain_of_thought':response_json['chain_of_thought'],
            'tables':candidate_table_names,
            'rewritten_question':rewriter_response}

# Example usage
user_question = "For all the transactions happened during 8:00-9:00 in 2012/8/26, how many happened in CZE?"

start_time = time.time()
result = execute_query_pipeline(user_question)
end_time = time.time()

latency = end_time - start_time
print("Result:", result)
print("Latency:", latency, "seconds")

Rewritten Question: What is the total count of transactions that occurred in the CZE location between 8:00 AM and 9:00 AM on August 26, 2012?
Candidate Tables: ['{\n    "name": "public.yearmonth",\n    "business_readable_name": "Customer Monthly Consumption",\n    "description": "The transactions_1k table records detailed information about individual transactions at gas stations, including the transaction date and time, customer identification, card used, gas station, and product details. It also includes the amount and price of products purchased, with the total price being the product of amount and price. This table is essential for analyzing sales activities, customer purchasing habits, and financial performance in the context of fuel and related product sales.",\n    "columns": [\n        {\n            "name": "customerid",\n            "type": "bigint",\n            "description": null,\n            "definition": "This column contains the unique identification numbers assigned to

In [15]:
import pandas as pd

df = pd.read_json('mini_dev_postgresql.json')
eval = df[df['db_id']=='debit_card_specializing']
eval.head()

,question_id,db_id,question,evidence,SQL,difficulty
0,1471,debit_card_specializing,What is the ratio of customers who pay in EUR ...,ratio of customers who pay in EUR against cust...,SELECT CAST(SUM(CASE WHEN Currency = 'EUR' THE...,simple
1,1472,debit_card_specializing,"In 2012, who had the least consumption in LAM?",Year 2012 can be presented as Between 201201 A...,SELECT T1.CustomerID FROM customers AS T1 INNE...,moderate
2,1473,debit_card_specializing,What was the average monthly consumption of cu...,Average Monthly consumption = AVG(Consumption)...,"SELECT AVG(T2.Consumption) / NULLIF(12, 0) FRO...",moderate
3,1476,debit_card_specializing,What was the difference in gas consumption bet...,cast the consumption into float when perform c...,SELECT SUM(CASE WHEN T1.Currency = 'CZK' THEN ...,challenging
4,1479,debit_card_specializing,Which year recorded the most consumption of ga...,The first 4 strings of the Date values in the ...,"SELECT SUBSTR(T2.Date, 1, 4) FROM customers AS...",moderate


In [39]:
from tqdm import tqdm

tqdm.pandas()

eval['llm_result'] = eval.progress_apply(lambda row: execute_query_pipeline(row['question'], row['evidence']), axis=1)
eval['generated_sql_query'] = eval['llm_result'].apply(lambda x: x['sql_query'])

  0%|          | 0/30 [00:00<?, ?it/s]

Rewritten Question: What is the ratio of the number of customers who paid in EUR to the number of customers who paid in CZK?
Candidate Tables: ['{\n    "name": "public.customers",\n    "business_readable_name": "Customer Segments",\n    "description": "The customers table contains information about individual customers, including a unique identifier, their client segment classification, and the currency they use for transactions. This table helps in distinguishing customers and categorizes them into different groups such as Key Account Management (KAM), Large Account Management (LAM), and Small and Medium Enterprises (SME). The currency column indicates the currency codes like CZK and EUR used by customers in their transactions. This information is used for tailoring services and performing financial analyses.",\n    "columns": [\n        {\n            "name": "customerid",\n            "type": "bigint",\n            "description": null,\n            "definition": "Customer ID is a un

  7%|▋         | 2/30 [00:04<01:01,  2.18s/it]

Generated Chain of Thought: To find the ratio of customers who pay in EUR against customers who pay in CZK, we need to count the number of customers using each currency. This information is located in the 'public.customers' table in the 'currency' column. We can use a SQL query to count the occurrences of each currency type and then compute the ratio of these counts.
Generated SQL Query: SELECT (SUM(CASE WHEN "currency" = 'EUR' THEN 1 ELSE 0 END)::decimal / NULLIF(SUM(CASE WHEN "currency" = 'CZK' THEN 1 ELSE 0 END), 0)) AS eur_to_czk_ratio FROM public.customers;
Rewritten Question: Which individual or entity had the lowest consumption in LAM during the year 2012, based on data from the yearmonth table?
Candidate Tables: ['{\n    "name": "public.yearmonth",\n    "business_readable_name": "Customer Monthly Consumption",\n    "description": "The transactions_1k table records detailed information about individual transactions at gas stations, including the transaction date and time, custom

 10%|█         | 3/30 [00:08<01:24,  3.15s/it]

...

Generated Chain of Thought: To determine the total amount spent by customer 38508 at the gas stations, we need to query the transactions_1k table. We will filter the transactions by the customer ID 38508. Next, to find out how much the customer spent in January 2012, we will need to use the same table but add a date filter to restrict the transactions to that specific month. The total amount spent can be calculated using the amount and price columns. We can combine these operations into a single query.
Generated SQL Query: SELECT SUM("amount" * "price") AS total_spent, SUM(CASE WHEN TO_CHAR("date", 'YYYYMM') = '201201' THEN "amount" * "price" ELSE 0 END) AS jan_2012_spent FROM transactions_1k WHERE "customerid" = 38508;
Rewritten Question: Who is the top spending customer, what is the average price per single item purchased by this customer, and what currency was used for these transactions?
Candidate Tables: ['{\n    "name": "public.customers",\n    "business_readable_name": "Customer

100%|██████████| 30/30 [02:16<00:00,  5.88s/it]

Generated Chain of Thought: To identify the top spending customer and calculate the average price per item purchased by this customer along with the currency used, we need to follow these steps:

1. Identify the total spending per customer to determine the top spender.
2. Once we identify the top spender, calculate the average price per item for this customer.
3. Get the currency used by this customer from the customers table.

We will need to join the transactions_1k table with the customers table to get the customer's currency.

Query 1: Calculate total spending per customer to identify the top spender.
Query 2: Calculate the average price per single item purchased by this customer.
Query 3: Identify the currency used by this customer.
Generated SQL Query: WITH total_spending AS (SELECT "customerid", SUM("price" * "amount") AS total_spent FROM transactions_1k GROUP BY "customerid" ORDER BY total_spent DESC LIMIT 1), 
 top_customer AS (SELECT ts."customerid", ts.total_spent, c."curren

100%|██████████| 30/30 [02:21<00:00,  4.73s/it]

Generated Chain of Thought: To answer this question, we need to follow these steps: 1) Identify the table that contains the transaction information including product ID, price, and customer ID. This information is in the 'transactions_1k' table. 2) Filter the transactions to get records where the price per unit is greater than 29.00 and product ID is 5. 3) From these records, extract the unique customer IDs. 4) Use these customer IDs to get the consumption status in August 2012 from the 'yearmonth' table.
Generated SQL Query: SELECT y."customerid", y."consumption" FROM public.yearmonth y WHERE y."date" = '201208' AND y."customerid" IN (SELECT t."customerid" FROM public.transactions_1k t WHERE t."price" / t."amount" > 29.00 AND t."productid" = 5)


In [40]:
# execute the queries against the database and output the results

def run_query(query):
    try:
        result = sql_client.query(query)
        return result
    except Exception as e:
        return str(e)
    
eval['generated_query_result'] = eval['generated_sql_query'].progress_apply(run_query)
eval['ground_truth_result'] = eval['SQL'].progress_apply(run_query)

100%|██████████| 30/30 [00:04<00:00,  7.30it/s]


In [56]:
class isValidSQL:
    def __init__(self, db_client: DatabaseClient):
        self.db_client = db_client

    def __call__(self,*, query,**kwds):
        try:
            result = self.db_client.query(query)
            return 1
        except Exception as e:
            return 0

class SQLExecutionAccuracy:
    def __init__(self, db_client: DatabaseClient, openai_client: AzureOpenAI):
        self.db_client = db_client

    def __call__(self,*, question, query, expected_result, **kwds):
        evaluator_prompt = f"""You are a helpful AI SQL expert.
        Your are given a question, the expected result and the results generated by a SQL query.
        Your role is to determine if the ground truth result and the generated result are the same.
        These are query results so the columns may not be in the same order or have the same names but contain the right information.


        ## Specifically you must:
        - Compare the expected result with the generated result
        - If the generated result is not the same as the expected result but contains the same information or numerical data, you return 1
        - If the generated result is exactly the same as the expected result, you return 1
        - If the generated result is different from the expected result and does not contain the same information, you return 0
        - If two numeric values are rounded differently, you should consider them the same

        You cannot return anything other than 1 or 0. Your answer must be an integer.
        
        ## Examples
        - Question: What is the average amount spent by customer 234?
        - Generated Result: [{{"average_spend": 0.06572769953051644}}]
        - Expecter Result: [{{"?column?": 0.06572769953051644}}]

        Answer: 1
        """
        try:
            result = self.db_client.query(query)
            user_prompt = f"""
                    Question: {question}
                    Generated Result: {result}
                    Expected Result: {expected_result}
                    """
            messages = [{"role": "system", "content": evaluator_prompt},
                        {"role": "user", "content": user_prompt},
                        ]
            response = openai_client.chat.completions.create(
                model="gpt-4o-global",
                messages=messages,
            )
            response = response.choices[0].message.content
            return int(response)
        except Exception as e:
            return None
                        

In [57]:

eval['IsValidSQL']=eval.progress_apply(lambda row: isValidSQL(sql_client)(query=row['generated_sql_query']), axis=1)
eval['SQLExecutionAccuracy'] = eval.progress_apply(lambda row: SQLExecutionAccuracy(sql_client, openai_client)(question=row['question'],
                                                                                                                query=row['generated_sql_query'],
                                                                                                                expected_result=row['ground_truth_result']), axis=1)

execution_accuracy = eval['SQLExecutionAccuracy'].mean()
valid_sql = eval['IsValidSQL'].mean()

print("SQL Execution Accuracy:", execution_accuracy)
print("Valid SQL Queries:", valid_sql)

100%|██████████| 30/30 [00:13<00:00,  2.27it/s]

SQL Execution Accuracy: 0.5925925925925926
Valid SQL Queries: 0.9
